# api

> Main entrypoint api for beetroot

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| default_exp api

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
from fastcore.test import *
from IPython.display import display, Markdown
import json
import os
from pathlib import Path
import re
import tempfile


In [ ]:
#| hide
#| export
import io
from typing import Callable, Dict, Iterable, Reversible, Sequence, Tuple

In [ ]:
# | export
from beetroot.source import (
    emit_markdown_source,
    emit_python_source,
    parse_and_extract_directives_from_python_source,
)
from beetroot.outputs import (
    Completion,
    emit_display_data_output,
    emit_execute_result_output,
    emit_stream_output,
)

In [ ]:
# | export


class Transformer:
    """Base class for all content transformers."""

    def emit_before(self, stream: io.TextIOBase):
        """Implement this method on sub-classes to emit markdown \
            before the set of lines this transformer processes."""
        return

    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        """Implement this method on sub-classes to modify the lines \
            this transformer processes."""
        return lines

    def emit_after(self, stream: io.TextIOBase):
        """Implement this method on sub-classes to emit markdown \
            after the set of lines this transformer processes."""
        return

In [ ]:
show_doc(Transformer.emit_before)

---

[source](https://github.com/spather/beetroot/blob/main/beetroot/api.py#L21){target="_blank" style="float:right; font-size:smaller"}

### Transformer.emit_before

>      Transformer.emit_before (stream:io.TextIOBase)

Implement this method on sub-classes to emit markdown             before the set of lines this transformer processes.

In [ ]:
show_doc(Transformer.process_lines)

---

[source](https://github.com/spather/beetroot/blob/main/beetroot/api.py#L24){target="_blank" style="float:right; font-size:smaller"}

### Transformer.process_lines

>      Transformer.process_lines (lines:Sequence[str])

Implement this method on sub-classes to modify the lines             this transformer processes.

In [ ]:
show_doc(Transformer.emit_after)

---

[source](https://github.com/spather/beetroot/blob/main/beetroot/api.py#L27){target="_blank" style="float:right; font-size:smaller"}

### Transformer.emit_after

>      Transformer.emit_after (stream:io.TextIOBase)

Implement this method on sub-classes to emit markdown             after the set of lines this transformer processes.

In [ ]:
# | exporti

def emit_with_transformations(
    transformers: Reversible[Transformer],
    lines: Sequence[str],
    emit_lines_func: Callable[[Sequence[str], io.TextIOBase], None],
    stream: io.TextIOBase,
):
    # Emit all the before output from the transformers.
    # Do it in reversed order so that the first transformer's 
    # before output appears closest to the (transformed) lines
    for transformer in reversed(transformers):
        transformer.emit_before(stream)

    # Pass the source lines through all the transformers
    for transformer in transformers:
        lines = transformer.process_lines(lines)

    # Emit the now transformed lines
    emit_lines_func(lines, stream)

    # Emit the after output from the transformers
    for transformer in transformers:
        transformer.emit_after(stream)


In [ ]:
# | hide
# Tests for emit_with_transformations()
class TransformerA(Transformer):
    def emit_before(self, stream: io.TextIOBase):
        stream.write('before A\n')

    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        return [
            line.replace('a', 'A')
            for line in lines 
        ]

    def emit_after(self, stream: io.TextIOBase):
        stream.write('after A\n')

class TransformerB(Transformer):
    def emit_before(self, stream: io.TextIOBase):
        stream.write('before B\n')

    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        return [
            line.replace('b', 'B')
            for line in lines 
        ]

    def emit_after(self, stream: io.TextIOBase):
        stream.write('after B\n')

def dummy_emit_lines(lines: Sequence[str], stream: io.TextIOBase):
    for line in lines:
        stream.write(line)

lines = [
    'abCDefab\n',
    'ghiAbajkl\n'
    'mnoAAAAAABBBBBp\n'
]

stream = io.StringIO()

emit_with_transformations(
    transformers=[TransformerA(), TransformerB()],
    lines=lines,
    emit_lines_func=dummy_emit_lines,
    stream=stream
)

stream.seek(0)
output = stream.read()

expected = """\
before B
before A
ABCDefAB
ghiABAjkl
mnoAAAAAABBBBBp
after A
after B
"""

test_eq(output, expected)

In [ ]:
# | export
def export_notebook(
    nb_json: Dict, transformers_map: Dict[str, Reversible[Transformer]] = {}
) -> Tuple[str, Iterable[Completion]]:
    stream = io.StringIO()
    completions = []
    for cell in nb_json['cells']:
        if cell['cell_type'] == 'markdown':
            emit_with_transformations(
                transformers_map.get('markdown/source', []),
                cell['source'],
                emit_markdown_source,
                stream,
            )
            stream.write('\n')
        elif cell['cell_type'] == 'code':
            python_source, directives = parse_and_extract_directives_from_python_source(
                cell['source']
            )

            # Handle directives per https://quarto.org/docs/reference/cells/cells-jupyter.html#code-output
            # and https://quarto.org/docs/reference/cells/cells-jupyter.html#cell-output.
            should_echo = 'echo' not in directives or directives['echo']
            should_show_output = 'output' not in directives or directives['output']

            if should_echo:
                emit_python_source(python_source, stream)
                stream.write('\n')

            if not should_show_output:
                continue

            for output in cell['outputs']:
                output_type = output['output_type']
                if output_type == 'stream':
                    emit_stream_output(output, stream)
                elif output_type == 'display_data':
                    completion = emit_display_data_output(output, stream)
                    if completion:
                        completions.append(completion)
                elif output_type == 'execute_result':
                    completion = emit_execute_result_output(output, stream)
                    if completion:
                        completions.append(completion)
                stream.write('\n')

    stream.seek(0)
    return stream.read(), completions

In [ ]:
# Form the path to the testcase notebook
nbpath = os.path.dirname(os.path.realpath('__file__')) # Get path to this notebook
test_nb_path = Path(nbpath) / '../test_artifacts' / 'testcase_notebook.ipynb'

# Load the testcase notebook as JSON
nb_json = json.loads(test_nb_path.read_text())

# Call export, get the text and the completions
md_text, completions = export_notebook(nb_json)

# Create a temporary directory we'll use for running the completions
with tempfile.TemporaryDirectory() as tmpdir:
    tmp_path = Path(tmpdir)
    for completion in completions:
        completion(tmp_path)
        
    filename = None 
    with open(tmp_path / 'markdown.md', 'w') as md_file:
        md_file.write(md_text)
        filename = md_file.name 

    # Images won't render because their paths are relative to the markdown file,
    # not this notebook. That's unfortunate but I've decided to live with it. 
    # The solution would be to temporarily copy images from the output directory
    # to a local images/ directory here. But that seems brittle so I'm not doing
    # it for now. 
    display(Markdown(filename=filename))

```python
name = 'world'
print(f"hello, {name}")
```


```
hello, world
```

# This is Markdown
This is a [link](https://www.google.com).
This is:

* a 
* bulleted
* list

Yup.

```python
# Multiline stream output
print("hello, world")
print("a second line")
print("a third line")
```


```
hello, world
a second line
a third line
```

```python
# Single line stream output with no trailing newline
import sys
sys.stdout.write("hello")
```


```
hello
```

```
5
```

```python
# Multiline stream output with no trailing newline
print("hello with a newline")
sys.stdout.write("hello")
```


```
hello with a newline
hello
```

```
5
```

```
This output should show but the source code shouldn't
```

```python
# The source code for this cell will show but its output won't 

print("This output should not show")
```


The cell below is intentionally left blank to test handling blank cells - do not remove it

```python

```


The above cell was intentionally left blank to test handling blank cells - do not remove it

```python
# Test latex output
from IPython.display import Latex

Latex('$\\displaystyle w_{11} x_{11} + w_{12} x_{12} + w_{21} x_{21} + w_{22} x_{22}$')
```


$\displaystyle w_{11} x_{11} + w_{12} x_{12} + w_{21} x_{21} + w_{22} x_{22}$

```python
# Test inline image
%matplotlib inline 
from matplotlib import pyplot as plt
from math import cos, radians, sin 

fig = plt.figure(figsize=(2, 1))
ax = fig.add_subplot(1, 1, 1)
ax.set(
    title="sin(x) and cos(x)"
)
xs = range(0,360)
ax.plot(xs, [sin(radians(x)) for x in xs], label="sin(x)")
ax.plot(xs, [cos(radians(x)) for x in xs], label="cos(x)")
_ = ax.legend()
```


![](images/4256d82d-45e8-4a8b-a05e-d650cdcc49ce.png)
```python

```




In [ ]:
# Form the path to the testcase notebook
nbpath = os.path.dirname(os.path.realpath('__file__'))  # Get path to this notebook
test_nb_path = (
    Path(nbpath) / '../test_artifacts' / 'transformations_testcases.ipynb'
)

# Load the testcase notebook as JSON
nb_json = json.loads(test_nb_path.read_text())


class ReplaceDollarSigns(Transformer):
    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        regex = r"\$(?!\$)(.*?[^\\])\$"
        replacement = r"\\\\(\1\\\\)"

        return [re.sub(regex, replacement, line) for line in lines]


# Call export, get the text and the completions
md_text, _ = export_notebook(
    nb_json, transformers_map={'markdown/source': [ReplaceDollarSigns()]}
)

Markdown(md_text)

This is some markdown in which I will replace the dollar signs in the following inline expression: \\(a + b\\).



In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()